<a href="https://colab.research.google.com/github/dnmuturi/SIT742/blob/main/SIT742Task2code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**END TERM ASSESSMENT**

 **GROUP C30 Members**

1. David Muturi - S225177509
2. Nhlanhla Matukane
3. Vincent Nwobi

### **Part 1**

**Answer 1.1**

In [19]:
#install the spark library
!pip install pyspark

In [20]:
#load necessary libraries
from google.colab import drive
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_unixtime, to_date, when, lit, trim
from google.colab import drive

In [21]:
# Mount Google Drive
drive.mount('/content/drive')
# Define the path to the CSV file
csv_path = '/content/drive/My Drive/SIT742/review.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
# Create a SparkSession
spark = SparkSession.builder.appName("ReadCSVCorrectly").getOrCreate()
# Read the CSV file into a PySpark DataFrame with options to improve parsing
try:
    df_review= spark.read.csv(
        csv_path,
        header=True,
        inferSchema=True,
        sep=',',  # Assuming comma is the delimiter
        quote='"', # Assuming double quotes are used for quoting fields
        escape='"', # Assuming double quotes are escaped by double quotes
        multiLine=True # Set to true if text column contains newline characters
    )
    print("CSV file loaded successfully!")
    df_review.show(5, truncate=False) # Display the first five rows without truncating
except Exception as e:
    print(f"An error occurred: {e}")

CSV file loaded successfully!
+---------------------+-----------------+-------------+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+----+-------------------------------------+
|user_id              |name             |time         |rating|text                                                

In [23]:
#display total no of rows
print(df_review.count())

521515


In [29]:
# show data tpes
df_review.printSchema()

root
 |-- user_id: decimal(21,0) (nullable = true)
 |-- name: string (nullable = true)
 |-- time: long (nullable = true)
 |-- rating: integer (nullable = true)
 |-- text: string (nullable = false)
 |-- pics: string (nullable = true)
 |-- resp: string (nullable = true)
 |-- gmap_id: string (nullable = true)
 |-- newtime: date (nullable = true)



In [31]:
# Show the schema of the df_review DataFrame to check the 'time' column data type
df_review.printSchema()

root
 |-- user_id: decimal(21,0) (nullable = true)
 |-- name: string (nullable = true)
 |-- time: long (nullable = true)
 |-- rating: integer (nullable = true)
 |-- text: string (nullable = false)
 |-- pics: string (nullable = true)
 |-- resp: string (nullable = true)
 |-- gmap_id: string (nullable = true)
 |-- newtime: date (nullable = true)



**Answer 1.1.1**

In [24]:
def show_empty_rows(df):

  # Filter rows where 'text' column is null or an empty string
  null_or_empty_text_count = df.filter(
    col("text").isNull() | (trim(col("text")) == "")
  ).count()
  return null_or_empty_text_count

In [25]:
#show count of empty rows in text field before filling in with no review
print(f"The number of empty rows in the text column is before replacing with no review is: {show_empty_rows(df_review)}")

The number of empty rows in the text column is before replacing with no review is: 223258


In [26]:

# Replace null values with "No review"
df_review = df_review.fillna({'text': 'No review'})

# Replace empty strings (after trimming whitespace) with "No review"
df_review = df_review.withColumn("text",
    when(trim(col("text")) == "", lit("no review")).otherwise(col("text"))
)

# Show the count of text column with  to verify the changes
print(f"The number of empty rows in the text column after replacing with no review is: {show_empty_rows(df_review)}")

The number of empty rows in the text column after replacing with no review is: 0


**Answer 1.1.2**

In [35]:
from pyspark.sql.functions import col, from_unixtime, to_date
from pyspark.sql import SparkSession
from pyspark.sql.types import LongType, DoubleType # Import DoubleType

# Ensure SparkSession is available and set the time parser policy
spark = SparkSession.builder.appName("ConvertTime").getOrCreate()
spark.conf.set("spark.sql.legacy.timeParserPolicy", "CORRECTED")

# Convert the 'time' column (in milliseconds) to seconds (DoubleType), then to timestamp, then to date
df_review = df_review.withColumn("newtime",
    to_date(from_unixtime(col("time").cast(DoubleType()) / 1000), "yyyy-MM-dd"))

# Show the first 5 rows with the new 'newtime' column
df_review.show(5)

+--------------------+-----------------+-------------+------+--------------------+----+----+--------------------+-------+
|             user_id|             name|         time|rating|                text|pics|resp|             gmap_id|newtime|
+--------------------+-----------------+-------------+------+--------------------+----+----+--------------------+-------+
|10912980484268600...|       Nicki Gore|1566331951619|     5|We always stay he...|NULL|NULL|0x56b646ed2220b77...|   NULL|
|11324092640575800...|    Allen Ratliff|1504917982385|     5|Great campground ...|NULL|NULL|0x56b646ed2220b77...|   NULL|
|11304483789114100...|Jonathan Tringali|1474765901185|     4|We tent camped he...|NULL|NULL|0x56b646ed2220b77...|   NULL|
|11032915514759200...|           S Blad|1472858535682|     4|This place is jus...|NULL|NULL|0x56b646ed2220b77...|   NULL|
|10898963490860200...|   Daniel Formoso|1529649811341|     5|Probably the nice...|NULL|NULL|0x56b646ed2220b77...|   NULL|
+--------------------+--

In [36]:
# Show the schema of the df_review DataFrame
df_review.printSchema()

# Show the first 5 rows of the df_review DataFrame
df_review.show(5)

root
 |-- user_id: decimal(21,0) (nullable = true)
 |-- name: string (nullable = true)
 |-- time: long (nullable = true)
 |-- rating: integer (nullable = true)
 |-- text: string (nullable = false)
 |-- pics: string (nullable = true)
 |-- resp: string (nullable = true)
 |-- gmap_id: string (nullable = true)
 |-- newtime: date (nullable = true)

+--------------------+-----------------+-------------+------+--------------------+----+----+--------------------+-------+
|             user_id|             name|         time|rating|                text|pics|resp|             gmap_id|newtime|
+--------------------+-----------------+-------------+------+--------------------+----+----+--------------------+-------+
|10912980484268600...|       Nicki Gore|1566331951619|     5|We always stay he...|NULL|NULL|0x56b646ed2220b77...|   NULL|
|11324092640575800...|    Allen Ratliff|1504917982385|     5|Great campground ...|NULL|NULL|0x56b646ed2220b77...|   NULL|
|11304483789114100...|Jonathan Tringali|1474